In [3]:
import os
import numpy as np
import cv2
import fnmatch
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from pathlib  import Path
import h5py

input_preds ='./'

In [4]:
files  = fnmatch.filter(os.listdir(f"{input_preds}/out_til/"), '*.png') #contour h5y files by create_patches_fn.py from pickles by /home/jupyter/src/Mask_annotation_jang.ipynb

with open('result_Tumor_tils_density_test.csv', 'w') as outfile :
    outfile.write('file_name,immune_infl,immune_excl,immune_des\n')
    for filename in files :
        slide_name = filename.split('.')[0] #.replace('.svs', '_gray.png')

        tumor = input_preds + '/out_tumor/' + filename
        til = input_preds + '/out_til/' + filename            
        stroma = input_preds + '/out_stroma/' + filename

        if not (os.path.exists(tumor) and os.path.exists(til) and os.path.exists(stroma)) : continue

        tumor_img_gray = cv2.imread(tumor, cv2.IMREAD_GRAYSCALE)
        til_img_gray = cv2.imread(til, cv2.IMREAD_GRAYSCALE)
        stroma_img_gray = cv2.imread(stroma, cv2.IMREAD_GRAYSCALE)

        til_img_gray[til_img_gray < 128] = 0
        til_img_gray[til_img_gray >= 128] = 1

        stroma_img_gray[stroma_img_gray < 128] = 0
        stroma_img_gray[stroma_img_gray >= 128] = 1

        tumor_img_gray = cv2.resize(tumor_img_gray, (til_img_gray.shape[1],til_img_gray.shape[0]))
        tumor_img_gray[tumor_img_gray < 128] = 0
        tumor_img_gray[tumor_img_gray >= 128] = 1

        out = np.zeros((tumor_img_gray.shape[0], tumor_img_gray.shape[1], 3))

        out[:,:,2] = tumor_img_gray #R
        out[:,:,1] = stroma_img_gray #G
        out[:,:,0] = til_img_gray #B

        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__tumor.png', tumor_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__stroma.png', stroma_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__til.png', til_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__comb.png', out*255)

        tissue_mask_img_gray = cv2.bitwise_or(til_img_gray, tumor_img_gray)
        tissue_mask_img_gray = cv2.bitwise_or(tissue_mask_img_gray, stroma_img_gray)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__mask.png', tissue_mask_img_gray*255)

        overlap_tt = til_img_gray + tumor_img_gray # overlap TIL and Tumor
        overlap_ts=til_img_gray + stroma_img_gray # overlap TIL and stroma
        overlap_tst=tissue_mask_img_gray + til_img_gray # overlap Tissue and TIL

        # binarize overlapped images
        c_overlap_tt = overlap_tt > 1 
        c_overlap_ts=overlap_ts > 1
        c_overlap_tst=overlap_tst > 1

        imm_inf=0
        imm_excl=0
        imm_des=0
        c_tissue= 0

        i=0

        M,N = 17,17
        
        tiles = [tissue_mask_img_gray[x:x+M,y:y+N] for x in range(0,tissue_mask_img_gray.shape[0],M) for y in range(0,tissue_mask_img_gray.shape[1],N)]
        rows, cols = tissue_mask_img_gray.shape
        img_color = np.dstack((tumor_img_gray, stroma_img_gray, til_img_gray)) * 255

        for x in range(0,tissue_mask_img_gray.shape[0],M):
            for y in range(0,tissue_mask_img_gray.shape[1],N):
                #print(i)
                #print(tiles[i].sum())
                if (tiles[i].sum())>0:
                    tissue_sum=tiles[i].sum()
                    c_tissue+=1 #count the number of tissue tiles
                    overlap_count_tt = sum(map(lambda x : x == True, c_overlap_tt[x:x+M,y:y+N]))#count the overlap ratio
                    overlap_count_ts = sum(map(lambda x : x == True, c_overlap_ts[x:x+M,y:y+N]))
                    overlap_count_tst = sum(map(lambda x : x == True, c_overlap_tst[x:x+M,y:y+N]))
                    sum_overlap_tt = np.sum(overlap_count_tt)
                    sum_overlap_ts = np.sum(overlap_count_ts)
                    sum_overlap_tst = np.sum(overlap_count_tst)
                    #print(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]),sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]),sum_overlap_tst)
                    if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.2:
                        imm_inf+=1                       
                        # start_point = (y, x - M)  topleft
                        # end_point = (y + N, x) +  bottomright
                        # color = (255, 0, 255) magenta
                        # thickness = 1
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (255, 0, 255), 1)
                        
                       
                    elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]) >0.1:
                        imm_excl+=1
                        # color = (255, 165, 0) orange
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (255, 165, 0), 1)


                    else:
                        imm_des+=1
                        # color = (0, 100, 1000) cyan
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (0, 255, 255), 1)

                i+=1
            
            
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__immune_sub.png', cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR))
                        
        if c_tissue == 0 : continue
        outfile.write(filename
                      + ',' + str(imm_inf/c_tissue)
                      + ',' + str(imm_excl/c_tissue)
                      + ',' + str(imm_des/c_tissue) + '\n')
    outfile.close()

/home/ext_shin_rumi_mayo_edu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in long_scalars
/home/ext_shin_rumi_mayo_edu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in long_scalars


<Figure size 432x288 with 0 Axes>

In [ ]:
def tils_density(input_path):
    tils_hm_files = fnmatch.filter(os.listdir(input_path+'pred_tils/'), '*Tumor*_gray.png')
    tils_hm_files.sort()
    M=17; N=17
    with open('result_Tumor_tils_density_test.csv', 'a') as outfile :
        outfile.write('file_name,immune_infl,immune_excl,immune_des\n')
        for tils_file in tils_hm_files :
            tissue_mask_file = tils_file.split('_gray')[0] + '.png'
            tumor_file = tils_file
            til = input_path + '/pred_tils/' + tils_file
            tissue_mask = input_path + '/tissue_mask/'+ tissue_mask_file
            tumor = input_path + '/pred_tumor/' + tumor_file
            stroma=input_path + '/pred_stromal/' + tils_file
            til_img = plt.imread(til)
            #plt.imshow(til_img)
            #plt.show()
            #print('til:',til_img.shape)
            R, G, B = til_img[:,:,0], til_img[:,:,1], til_img[:,:,2]
            til_img_gray=np.mean([R,G,B], axis=0)
            til_img_gray[til_img_gray >= 0.5] = 1
            til_img_gray[til_img_gray < 0.5] = 0
            tumor_img = plt.imread(tumor)
            #print('Tumor:',tumor_img.shape)
            #plt.imshow(tumor_img)
            #plt.show()
            R, G, B = tumor_img[:,:,0], tumor_img[:,:,1], tumor_img[:,:,2]
            tumor_img_gray = np.mean([R,G,B], axis=0)
            tumor_img_gray[tumor_img_gray >= 0.5] = 1
            tumor_img_gray[tumor_img_gray < 0.5] = 0
            tumor_img_gray = cv2.resize(tumor_img_gray, (til_img_gray.shape[0],til_img_gray.shape[1]))
            tissue_mask_img = plt.imread(tissue_mask)
            #print('tissue:',tissue_mask_img.shape)
            #plt.imshow( tissue_mask_img)
            #plt.show()
            tissue_mask_img = cv2.resize(tissue_mask_img, (til_img.shape[0],til_img.shape[1]))
            R, G, B = tissue_mask_img[:,:,0], tissue_mask_img[:,:,1], tissue_mask_img[:,:,2]
            tissue_mask_img_gray = np.mean([R,G,B], axis=0)
            tissue_mask_img_gray[tissue_mask_img_gray >= 0.5] = 1
            tissue_mask_img_gray[tissue_mask_img_gray < 0.5] = 0
            tiles = [tissue_mask_img_gray[x:x+M,y:y+N] for x in range(0,tissue_mask_img_gray.shape[0],M) for y in range(0,tissue_mask_img_gray.shape[1],N)]
            #print(tiles)
            stroma_img = plt.imread(stroma)
            stroma_img = cv2.resize(stroma_img, (til_img_gray.shape[0],til_img_gray.shape[1]))
            R, G, B = stroma_img[:,:,0], stroma_img[:,:,1], stroma_img[:,:,2]
            stroma_img = np.mean([R,G,B], axis=0)
            #print('stroma:',stroma_img.shape)
            #plt.imshow(stroma_img)
            #plt.show()
            til_img_gray=np.reshape(til_img_gray,(tissue_mask_img_gray.shape))
            overlap_tt = til_img_gray + tumor_img_gray #overlap TIL and Tumor
            overlap_ts=til_img_gray + stroma_img#overlap of til and stroma
            overlap_tst=tissue_mask_img_gray + til_img_gray
            c_overlap_tt = overlap_tt > 1 #binarize
            c_overlap_ts=overlap_ts > 1
            c_overlap_tst=overlap_tst > 1
            imm_inf=0
            imm_excl=0
            imm_des=0
            c_tissue=0
            i=0
            for x in range(0,tissue_mask_img_gray.shape[0],M):
                for y in range(0,tissue_mask_img_gray.shape[1],N):
                    #print(i)
                    #print(tiles[i].sum())
                    if (tiles[i].sum())>100:
                        tissue_sum=tiles[i].sum()
                        c_tissue+=1 #count the number of tissue tiles
                        overlap_count_tt = sum(map(lambda x : x == True, c_overlap_tt[x:x+M,y:y+N]))#count the overlap ratio
                        overlap_count_ts = sum(map(lambda x : x == True, c_overlap_ts[x:x+M,y:y+N]))
                        overlap_count_tst = sum(map(lambda x : x == True, c_overlap_tst[x:x+M,y:y+N]))
                        sum_overlap_tt = np.sum(overlap_count_tt)
                        sum_overlap_ts = np.sum(overlap_count_ts)
                        sum_overlap_tst = np.sum(overlap_count_tst)
                        #print(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]),sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]),sum_overlap_tst)
                        if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.4:
                            imm_inf+=1
                        elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]) >0.2:
                            imm_excl+=1
                        else:
                            imm_des+=1
                    i+=1
            print(tissue_mask_file.split('_')[0],imm_inf,imm_excl,c_tissue,imm_des)
            outfile.write(tissue_mask_file.split('_')[0]
                              + ',' + str(imm_inf/c_tissue)
                              + ',' + str(imm_excl/c_tissue)
                              + ',' + str(imm_des/c_tissue) + '\n')
    outfile.close()
input_path = "/home/ext_jang_inyeop_mayo_edu/hwangt-share/post_process/VanAbel_Oropharynx_Tumor_TIL_Stroma/Van Abel-K_req32469_OPX TILs HE/"
tils_density(input_path)

In [5]:
import openslide 
from PIL import Image
svs = openslide.OpenSlide('./test_wsi/0_S_14214776_0.svs')
thumbnail = svs.get_thumbnail(svs.level_dimensions[2])
thumbnail.save('./test_wsi/0_S_14214776_0.png')

TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
